# crypto token correllation Heatmap

In [1]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
load_dotenv()  # Load environment variables from .env

api_key = os.getenv("GECKO_API")  # Get API key from .env
coin_id = "bitcoin"
vs_currency = "usd"
days = "30"

url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
params = {
    "vs_currency": vs_currency,
    "days": days
}
headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": api_key
}

response = requests.get(url, headers=headers, params=params)
print(response.text)

{"prices":[[1753182191591,118785.81023179082],[1753185898429,119182.68891245346],[1753189494311,119359.72945155788],[1753193165038,118703.26838071554],[1753196904039,119002.50760809887],[1753200557180,119010.09713031474],[1753204068147,118754.69781442013],[1753207731196,119184.9420945347],[1753211082563,119733.93468019101],[1753214734817,119302.45718960169],[1753218500996,119759.51655055693],[1753222129749,119647.13610350236],[1753225447098,119905.34412932584],[1753229080480,119958.86217219132],[1753232769683,119683.19953786394],[1753236029496,119583.73597101556],[1753239787978,119127.28034544508],[1753243445556,118687.14241553268],[1753247314993,118900.1656138629],[1753250916877,118701.95317626116],[1753254236433,118473.90255623346],[1753258105507,118379.73883290826],[1753261746854,118433.78513026409],[1753265319482,118078.32575499937],[1753268534329,118373.06712224887],[1753272275592,118586.26472420427],[1753275966616,118102.75685806658],[1753279294593,117604.01001209718],[1753283365

In [5]:
data = response.json()

# Convert 'prices' to DataFrame
df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

print(df.head())

                timestamp          price
0 2025-07-22 11:03:11.591  118785.810232
1 2025-07-22 12:04:58.429  119182.688912
2 2025-07-22 13:04:54.311  119359.729452
3 2025-07-22 14:06:05.038  118703.268381
4 2025-07-22 15:08:24.039  119002.507608


# Create Function

In [8]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import joblib


load_dotenv()  # Load environment variables from .env

api_key = os.getenv("GECKO_API")  # Get API key from .env
coin_id = "bitcoin"
vs_currency = "usd"
days = "30"



def fetch_token_price_history(token_ids, vs_currency="usd", days="30"):
    """
    Fetch historical price data for multiple tokens from CoinGecko Pro API.
    
    Args:
        token_ids (list): List of token ids (e.g., ['bitcoin', 'ethereum']).
        vs_currency (str): The target currency (default 'usd').
        days (str): Number of days to fetch (default '30').
        
    Returns:
        pd.DataFrame: DataFrame with timestamps as index and tokens as columns.
    """
    
    load_dotenv()
    api_key = os.getenv("GECKO_API")
    price_dfs = []

    for coin_id in token_ids:
        url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
        params = {
            "vs_currency": vs_currency,
            "days": days
        }
        headers = {
            "accept": "application/json",
            "x-cg-pro-api-key": api_key
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['timestamp', coin_id])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df.resample('1h').last()  # or '1D' for daily
        price_dfs.append(df)
        

    # Merge all dataframes on timestamp
    result_df = pd.concat(price_dfs, axis=1)
    return result_df
    

In [9]:
# Example: Get tokens from user input
tokens = input("Enter token IDs separated by commas (e.g., bitcoin,ethereum,solana): ").split(",")
tokens = [t.strip() for t in tokens]

df_prices = fetch_token_price_history(tokens, days="30")
print(df_prices.head())

# Save with joblib if needed
save_dir = "/Users/olaoluwatunmise/token-heatmap/Database"
os.makedirs(save_dir, exist_ok=True)
joblib.dump(df_prices, os.path.join(save_dir, "df_prices.joblib"))

                           bitcoin     ethereum      solana
timestamp                                                  
2025-07-22 11:00:00  118785.810232  3686.683641  198.546682
2025-07-22 12:00:00  119182.688912  3706.154262  199.968792
2025-07-22 13:00:00  119359.729452  3703.359014  202.323265
2025-07-22 14:00:00  118703.268381  3662.805579  200.004531
2025-07-22 15:00:00  119002.507608  3683.052743  200.928303


PermissionError: [WinError 5] Access is denied: '/Users/olaoluwatunmise'